In [ ]:
#Mount your drive account into colab

from google.colab import drive
drive.mount('/content/drive/')

In [0]:
#The machine settings are listed.

from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 13546188213017850192, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 788120060344331508
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 7375896109398190523
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11281553818
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 13080467798599184865
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [0]:
#Check the activated GPU device on remote machine

import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
#Change the directory to your workspace

import os
os.chdir('/content/drive/My Drive/gru_word2vec_tweets_classification/')

In [0]:
!pwd

/content/drive/My Drive/gru_word2vec_tweets_classification


In [0]:
#Install  tenserflow,keras,scikit-learn and gensim in remote machine

!pip install tensorflow
!pip install -q keras
!pip install scikit-learn
!pip install gensim==3.5.0

    100% |████████████████████████████████| 23.5MB 1.6MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [0]:
#Import modules & set up logging

In [0]:
import numpy as np
import pandas as pd

In [0]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Input, Dense, GRU, Embedding, CuDNNGRU
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras.callbacks import ModelCheckpoint
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, f1_score, confusion_matrix
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix

In [0]:
import gensim, logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import time

from gensim.models.word2vec import Word2Vec
from glob import glob

In [0]:
#Data load

tweet_list = []
label_list = []

for index, file in enumerate(glob('3000tweet/raw_texts/**/*.txt', recursive=True)):                                                                 # labels
    label_list.append(file.split("/")[-2]),                                                                 # labels
    tweet_list.append((open(file, encoding="windows-1254").read().replace('\n', ' ').strip().lower()))      # tweets

print("3000 tweet  :  ", len(tweet_list))
print("3000 label  :  ", len(label_list))


3000 tweet  :   3000
3000 label  :   3000


In [0]:
print("sum data : ",len(label_list))
value1 = [i for i in label_list if i in '1']
value2 = [i for i in label_list if i in '2']
value3 = [i for i in label_list if i in '3']
print("1.class : ",len(value1))
print("2.class : ",len(value2))
print("3.class : ",len(value3))

sum data :  3000
1.class :  756
2.class :  1287
3.class :  957


In [0]:
tweet_list[0]

'hayat paylaşınca güzel ;)'

In [0]:
#Preprocessing

#import Stemmer
import re


def splitIntoStem(message):

    
    return [removeNumeric(stripEmoji(singleCharacterRemove(removePunctuation
                                                           (removeHyperlinks
                                                            (removeHashtags
                                                             (removeUsernames
                                                              (stemWord(word)))))))) for word in message.split()]
def stemWord(tweet):

    ################################## for turkish   => !pip install snowballstemmer
    #stemmer = Stemmer.Stemmer('turkish')                      
    #return stemmer.stemWord(tweet).lower()
    
    return tweet.lower()

#Remove usernames
def removeUsernames(tweet):
    return re.sub('@[^\s]+', '', tweet)


#Remove hashtag
def removeHashtags(tweet):
    return re.sub(r'#[^\s]+', '', tweet)

#Remove link
def removeHyperlinks(tweet):
    return re.sub('((www\.[^\s]+)|(https?://[^\s]+))', '', tweet)

#Remove numeric character
def removeNumeric(value):
    blist2 = [item for item in value if not item.isdigit()]
    blist3 = "".join(blist2)
    return blist3

#Remove punctuation
def removePunctuation(tweet):

    return re.sub(r'[^\w\s]','',tweet)

#Remove single character
def singleCharacterRemove(tweet):
    return re.sub(r'(?:^| )\w(?:$| )', ' ', tweet)

#Remove emoji
def stripEmoji(text):

    RE_EMOJI = re.compile('[\U00010000-\U0010ffff]', flags=re.UNICODE)
    return RE_EMOJI.sub(r'', text)

In [0]:
for i,k in enumerate(tweet_list):
    tweet_list[i] = " ".join(splitIntoStem(k)).split()

In [0]:
tweet_list[0]

['hayat', 'paylaşınca', 'güzel']

In [0]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
#Remove stopwords

from nltk.corpus import stopwords


def removeStopWords(tweet_list):
    
    filtered_stopwords = []
    filtered_stopwords_list = []
    
    stop_words = stopwords.words('turkish')

    stop_words.append("bir")
    stop_words.append("iki")
    stop_words.append("üç")
    stop_words.append("dört")
    stop_words.append("beş")
    stop_words.append("altı")
    stop_words.append("yedi")
    stop_words.append("sekiz")
    stop_words.append("dokuz")
    stop_words.append("on")
    
    print("stop_words : ",stop_words)
    
    for i in tweet_list:
        filtered_sentence = [w for w in i if not w in stop_words]
        filtered_stopwords_list.append(filtered_sentence)                         #return list value
        filtered_stopwords.append(" ".join(filtered_sentence))                    #return string value
    
    return filtered_stopwords,filtered_stopwords_list                                      


tweet_list,filtered_stopwords_list = removeStopWords(tweet_list)

stop_words :  ['acaba', 'ama', 'aslında', 'az', 'bazı', 'belki', 'biri', 'birkaç', 'birşey', 'biz', 'bu', 'çok', 'çünkü', 'da', 'daha', 'de', 'defa', 'diye', 'eğer', 'en', 'gibi', 'hem', 'hep', 'hepsi', 'her', 'hiç', 'için', 'ile', 'ise', 'kez', 'ki', 'kim', 'mı', 'mu', 'mü', 'nasıl', 'ne', 'neden', 'nerde', 'nerede', 'nereye', 'niçin', 'niye', 'o', 'sanki', 'şey', 'siz', 'şu', 'tüm', 've', 'veya', 'ya', 'yani', 'bir', 'iki', 'üç', 'dört', 'beş', 'altı', 'yedi', 'sekiz', 'dokuz', 'on']


In [0]:
tweet_list[0]

'hayat paylaşınca güzel'

In [0]:
filtered_stopwords_list[0]

['hayat', 'paylaşınca', 'güzel']

In [0]:
import pandas as pd 

daf = pd.DataFrame({"tweet":tweet_list,"label":label_list})
daf.head()

,label,tweet
0,1,hayat paylaşınca güzel
1,1,hayat kesinlikle paylaşınca güzel
2,1,beni unutmayan tek adam turkcell
3,1,ozaman senin karsına cıkan denyoların türkcell...
4,1,manganın anda turkcell internetinde verdigi ko...


In [0]:
daf.iloc[0][1]

'hayat paylaşınca güzel'

In [0]:
target = daf['label'].values.tolist()
data = daf['tweet'].values.tolist()

In [0]:
x_train, x_test, y_train, y_test = \
    train_test_split(data,target, test_size=0.2)  # random split

In [0]:
print("x_train :",len(x_train))
print("x_test :",len(x_test))

x_train : 2400
x_test : 600


In [0]:
x_train[800]

'benim sorduğunuz tahmin sorularına yaklaşamayışım olacak kadar kötü tahmin yapan başkası var'

In [0]:
y_train[800]

'3'

In [0]:
tokenizer = Tokenizer()             #Tokenizer(num_words=5000) => 5000 words of the highest frequency
tokenizer.fit_on_texts(data)
tokenizer 

In [0]:
list(tokenizer.word_index)[0:10]

['turkcell',
 'mesaj',
 'güzel',
 'bi',
 'hayat',
 'ben',
 'var',
 'paylaşınca',
 'superonline',
 'mi']

In [0]:
print("len(tokenizer) :",len(list(tokenizer.word_index)))

len(tokenizer) : 11905


In [0]:
#Training set token with text_to_sequences.

x_train_tokens = tokenizer.texts_to_sequences(x_train)

In [0]:
x_train[800]

'benim sorduğunuz tahmin sorularına yaklaşamayışım olacak kadar kötü tahmin yapan başkası var'

In [0]:
print(x_train_tokens[800])

[41, 7300, 547, 7301, 7302, 264, 15, 292, 547, 298, 1499, 7]


In [0]:
x_test_tokens = tokenizer.texts_to_sequences(x_test)

In [0]:
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)

In [0]:
num_tokens[800]

12

In [0]:
np.max(num_tokens)

21

In [0]:
np.argmax(num_tokens)

1104

In [0]:
x_train[np.argmax(num_tokens)]

'kuzum bugun msj hakkım bıttı turkcel son dakka gol caktı bana msn ye gel gır gır yapalım ozledım sen samaya yapmayı'

In [0]:
#You must equal your comments length to use rnn

In [0]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

18

In [0]:
#How many of the eighteen length comments are included?

np.sum(num_tokens < max_tokens) / len(num_tokens)

0.9783333333333334

In [0]:
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens)

In [0]:
#Zero is added before the values given in the padding operation.

print("x_train_tokens :",x_train_tokens[0])
print("x_train_pad :",x_train_pad[0])

x_train_tokens : [394, 26, 1233, 9268, 656, 9269]
x_train_pad : [   0    0    0    0    0    0    0    0    0    0    0    0  394   26
 1233 9268  656 9269]


In [0]:
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens)

In [0]:
print("x_test_tokens :",x_test_tokens[0])
print("x_train_pad :",x_test_pad[0])

x_test_tokens : [6200, 6201, 6202]
x_train_pad : [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 6200 6201 6202]


In [0]:
print("x_train_pad.shape :",x_train_pad.shape)
print("x_train_pad.shape :",x_test_pad.shape)

x_train_pad.shape : (2400, 18)
x_train_pad.shape : (600, 18)


In [0]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

In [0]:
def tokens_to_string(tokens):
    words = [inverse_map[token] for token in tokens if token!=0]
    text = ' '.join(words)
    return text

In [0]:
x_train[800]

'benim sorduğunuz tahmin sorularına yaklaşamayışım olacak kadar kötü tahmin yapan başkası var'

In [0]:
print(x_train_tokens[800])

[41, 7300, 547, 7301, 7302, 264, 15, 292, 547, 298, 1499, 7]


In [0]:
tokens_to_string(x_train_tokens[800])

'benim sorduğunuz tahmin sorularına yaklaşamayışım olacak kadar kötü tahmin yapan başkası var'

In [0]:
#Embedding operations

In [0]:
#Save word2vec format (not binary)

"""
model = Word2Vec(filtered_stopwords_list)
model_save_location = "3000tweets_notbinary"
model.wv.save_word2vec_format(model_save_location)
"""

In [0]:
model = Sequential()

In [0]:
embedding_size = 100 

In [0]:
#Word2vec load(2.option) example

word2vec = {}
with open('3000tweets_notbinary', encoding='UTF-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vec = np.asarray(values[1:], dtype='float32')
        word2vec[word] = vec

In [0]:
print("x_test[0] :",x_test[40])
print("x_test_pad[0] :",x_test_pad[0])

x_test[0] : lanet olası turkcell li olarak vallahi aynı nağmelerden sıkıldım lan üşenmesem sırf cıvık reklam geçicem
x_test_pad[0] : [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 6200 6201 6202]


In [0]:
num_words = len(list(tokenizer.word_index))

In [0]:
embedding_matrix = np.random.uniform(-1, 1, (num_words, embedding_size))
for word, i in tokenizer.word_index.items():
    if i < num_words:
        embedding_vector = word2vec.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [0]:
embedding_matrix.shape

(11905, 100)

In [0]:
embedding_matrix[3]

array([ 0.08986612, -0.00983265, -0.06834069,  0.21036609,  0.12450906,
        0.10021846, -0.14543779, -0.05969704,  0.01708784,  0.00408125,
        0.11804756,  0.13679515, -0.08721917, -0.18287565, -0.04086864,
        0.11032961, -0.05940395, -0.10572232, -0.01191167, -0.11670027,
        0.04507362, -0.08680133,  0.06382117,  0.11812145,  0.06167458,
       -0.09334848, -0.0218655 , -0.15539245, -0.2001535 , -0.09700425,
       -0.03025606,  0.10673232, -0.07413075, -0.12416601, -0.06571722,
        0.15431167,  0.03929737,  0.09869193,  0.17016847, -0.08498783,
       -0.13190085, -0.11170611,  0.20067932, -0.01707822,  0.13537587,
       -0.02854906, -0.25869036, -0.2436277 , -0.13429926, -0.10332574,
       -0.13679075,  0.01509539, -0.11398716,  0.10181049,  0.23719978,
       -0.07496728,  0.04676896,  0.04866217, -0.31362775,  0.04191856,
        0.11699729, -0.15392013,  0.11405101,  0.09320572, -0.06823581,
       -0.11526767,  0.09690311,  0.19448787,  0.04655581, -0.03

In [0]:
#Model add word2vec embedding

model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    weights= [embedding_matrix],
                    input_length=max_tokens,        
                    trainable=True,              #the layer is trained
                    name='embedding_layer'))

In [0]:
#Model add random embedding (not word2vec, fasttext vs.)

"""
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='embedding_layer'))                   
"""

In [0]:
model.add(CuDNNGRU(units=16, return_sequences=True))
model.add(CuDNNGRU(units=8, return_sequences=True))
model.add(CuDNNGRU(units=4))
model.add(Dense(1, activation='softmax'))  


In [0]:
optimizer = Adam(lr=1e-3)

In [0]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 18, 100)           1190500   
_________________________________________________________________
cu_dnngru (CuDNNGRU)         (None, 18, 16)            5664      
_________________________________________________________________
cu_dnngru_1 (CuDNNGRU)       (None, 18, 8)             624       
_________________________________________________________________
cu_dnngru_2 (CuDNNGRU)       (None, 4)                 168       
_________________________________________________________________
dense (Dense)                (None, 1)                 5         
Total params: 1,196,961
Trainable params: 1,196,961
Non-trainable params: 0
_________________________________________________________________


In [0]:
for i,j in enumerate(y_train):
  y_train[i]=int(j)

In [0]:
model.fit(x_train_pad, y_train, epochs=10, batch_size=256)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
2400/2400 [==============================] - 3s 1ms/step - loss: nan - acc: 0.2521
Epoch 2/10
2400/2400 [==============================] - 0s 73us/step - loss: nan - acc: 0.2521
Epoch 3/10
2400/2400 [==============================] - 0s 76us/step - loss: nan - acc: 0.2521
Epoch 4/10
2400/2400 [==============================] - 0s 72us/step - loss: nan - acc: 0.2521
Epoch 5/10
2400/2400 [==============================] - 0s 73us/step - loss: nan - acc: 0.2521
Epoch 6/10
2400/2400 [==============================] - 0s 76us/step - loss: nan - acc: 0.2521
Epoch 7/10
2400/2400 [==============================] - 0s 75us/step - loss: nan - acc: 0.2521
Epoch 8/10
2400/2400 [==============================] - 0s 75us/step - loss: nan - acc: 0.2521
Epoch 9/10
2400/2400 [==============================] - 0s 73us/step - loss: nan - acc: 0.2521
Epoch 10/10
2400/2400 [==============================] - 0s 72us/step - loss: nan - acc: 0.2521
